In [5]:
import os
from PIL import Image
import numpy as np
from keras import backend as K
from keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import cv2
from keras.preprocessing.image import ImageDataGenerator

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


In [6]:
# img_data = os.listdir('./data/small_data/train')
img_data = os.listdir('./data/small_data/train')

flower_mapping = {'daisy':0, 'dandelion':1, 'rose':2, 'sunflower':3, 'tulip':4}

IMAGE_DIMS = (240, 320, 3)
data = []
labels = []


# loop over the input images
# for imagePath in imagePaths:
# 	# load the image, pre-process it, and store it in the data list
# 	image = cv2.imread(imagePath)
# 	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
# 	image = img_to_array(image)
# 	images.append(image)
 
# 	# extract the class label from the image path and update the
# 	# labels list
# 	label = imagePath.split(os.path.sep)[-2]
# 	labels.append(label)

for img_dir in img_data:
    for fn in os.listdir(os.path.join('./data/small_data/train/',img_dir)):
        fd = os.path.join('./data/small_data/train/',img_dir,fn)
        image = cv2.imread(fd)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = img_to_array(image)
        data.append(image)
        labels.append(flower_mapping.get(img_dir))
        
# for img_dir in img_data:
#     for fn in os.listdir(os.path.join('./data/image_data/train/',img_dir)):
#         fd = os.path.join('./data/image_data/train/',img_dir,fn)
#         image = cv2.imread(fd)
#         image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
#         image = img_to_array(image)
#         data.append(image)
#         labels.append(flower_mapping.get(img_dir))

In [7]:
data[0].shape

(240, 320, 3)

In [8]:
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [9]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

[INFO] data matrix: 180.00MB


In [10]:
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)

In [12]:
trainX = next(data_generator.flow(trainX, shuffle=False))
trainY = next(data_generator.flow(trainY, shuffle=False))


ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (80,))

In [177]:
EPOCHS = 20
INIT_LR = 1e-3
BS = 32

In [178]:
testX.shape

(565, 240, 320, 3)

In [179]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [180]:
##V0 CNN

# from keras.models import Sequential
# from keras.layers import Dense, Dropout, Activation, Flatten
# from keras.layers import Conv2D, MaxPooling2D
# from keras.optimizers import RMSprop, Adam

# num_classes = 5
# testY = keras.utils.to_categorical(testY, num_classes)
# trainY = keras.utils.to_categorical(trainY, num_classes)

# model = Sequential()
# model.add(Conv2D(16, (5, 5), padding='same',
#                  input_shape=trainX.shape[1:]))
# model.add(Activation('relu'))
# model.add(Conv2D(16, (5, 5)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (5, 5), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(32, (5, 5)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))
# model.summary()

##SmallVGGNet
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense

classes = 5
model = Sequential()
inputShape = trainX.shape[1:]
chanDim = -1
testY = keras.utils.to_categorical(testY, num_classes)
trainY = keras.utils.to_categorical(trainY, num_classes)
inputShape = trainX.shape[1:]

model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(64, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv2D(128, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(classes))
model.add(Activation("sigmoid"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_65 (Conv2D)           (None, 240, 320, 32)      896       
_________________________________________________________________
activation_94 (Activation)   (None, 240, 320, 32)      0         
_________________________________________________________________
batch_normalization_25 (Batc (None, 240, 320, 32)      128       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 80, 106, 32)       0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 80, 106, 32)       0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 80, 106, 64)       18496     
_________________________________________________________________
activation_95 (Activation)   (None, 80, 106, 64)       0         
__________

In [181]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
history = model.fit(trainX, trainY,
                    batch_size=4,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(testX, testY))
score = model.evaluate(testX, testY, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 2258 samples, validate on 565 samples
Epoch 1/20
2258/2258 [==============================] - 1445s 640ms/step - loss: 1.7634 - acc: 0.2750 - val_loss: 1.7708 - val_acc: 0.2903
Epoch 2/20
2258/2258 [==============================] - 1444s 640ms/step - loss: 1.5800 - acc: 0.3339 - val_loss: 1.5242 - val_acc: 0.3664
Epoch 3/20
2258/2258 [==============================] - 1430s 633ms/step - loss: 1.5461 - acc: 0.3299 - val_loss: 1.5198 - val_acc: 0.3965
Epoch 4/20
2258/2258 [==============================] - 1393s 617ms/step - loss: 1.5154 - acc: 0.3601 - val_loss: 1.4611 - val_acc: 0.4319
Epoch 5/20
2258/2258 [==============================] - 1435s 635ms/step - loss: 1.4839 - acc: 0.3835 - val_loss: 1.8296 - val_acc: 0.2867
Epoch 6/20
2258/2258 [==============================] - 1438s 637ms/step - loss: 1.5056 - acc: 0.3694 - val_loss: 1.5324 - val_acc: 0.3681
Epoch 7/20
2258/2258 [==============================] - 1395s 618ms/step - loss: 1.5964 - acc: 0.3273 - val_loss: 1.585

In [182]:
test_img_data = os.listdir('./data/image_data/test')
test_data = []
for test_img_dir in test_img_data:
    fd = os.path.join('./data/image_data/test/',test_img_dir)
    image = cv2.imread(fd)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    test_data.append(image)
test_data[0].shape

(240, 320, 3)

In [183]:
test_data = np.array(test_data, dtype="float") / 255.0
print("[INFO] data matrix: {:.2f}MB".format(
	test_data.nbytes / (1024 * 1000.0)))

[INFO] data matrix: 3600.00MB


In [184]:
ynew = model.predict(test_data)

In [185]:
ynew

array([[0.9970715 , 0.9978508 , 0.99999654, 0.9998927 , 1.        ],
       [0.418367  , 0.99977905, 0.9795855 , 0.9953295 , 0.9994764 ],
       [0.00391951, 0.05201413, 0.99974126, 0.94698113, 0.99999976],
       ...,
       [0.9999856 , 0.99998116, 0.03284308, 0.00807271, 0.9849975 ],
       [0.99588263, 0.999972  , 0.9891473 , 0.9996394 , 0.9979873 ],
       [0.97290516, 0.99966574, 0.99120456, 0.18990047, 0.94132465]],
      dtype=float32)

In [186]:
res = []
for yn in ynew:
    res.append(np.argmax(yn))
for r in res:
    print(r)

4
1
4
4
4
2
1
4
4
1
4
1
4
1
0
0
4
4
1
1
4
4
4
1
4
4
1
4
1
4
1
4
1
1
0
1
1
0
1
4
4
1
1
4
1
4
0
1
2
1
1
4
1
1
1
0
2
1
1
4
1
4
3
0
4
4
0
4
1
4
4
1
0
1
1
1
1
1
4
1
4
0
1
2
4
1
2
0
1
4
1
4
0
1
1
1
1
4
1
1
3
4
1
1
0
2
1
1
1
1
1
4
4
1
1
0
1
1
4
1
2
1
1
1
1
1
3
0
1
1
1
0
0
3
2
1
1
4
4
1
4
1
1
0
1
2
1
4
1
1
1
1
1
4
2
0
4
1
2
4
1
0
1
1
0
4
1
4
0
1
1
1
2
1
1
4
4
1
1
1
4
4
4
4
4
1
4
4
1
4
4
1
1
0
1
1
1
1
1
0
4
0
3
4
0
1
1
4
4
1
1
1
4
4
0
2
4
1
0
1
4
0
1
1
0
4
4
3
1
1
1
4
4
1
0
4
0
1
1
1
4
1
0
1
1
1
1
1
0
4
1
4
2
1
4
0
1
4
0
4
1
0
1
1
1
1
4
4
1
4
4
1
1
1
4
1
0
1
4
4
2
1
2
1
1
1
1
0
0
1
1
0
1
2
0
1
1
4
1
1
4
3
4
1
1
4
1
0
1
1
1
1
4
1
1
3
0
1
1
1
4
0
1
1
0
1
0
4
4
1
1
1
1
1
4
0
4
1
4
1
1
1
0
4
3
1
0
1
4
1
4
2
1
1
1
3
1
1
4
1
0
0
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
0
4
1
1
1
1
2
1
0
1
1
1
0
1
1
1
1
0
4
1
0
1
1
1
1
1
4
0
4
1
1
3
1
4
0
1
4
2
1
1
1
4
1
1
1
1
1
0
1
4
1
1
0
1
1
1
1
4
0
1
1
1
1
1
2
0
1
1
0
1
3
4
1
4
1
1
0
2
1
1
4
1
2
0
1
4
1
4
1
1
4
2
4
0
4
1
1
1
0
1
3
1
1
2
4
4
1
3
0
1
4
1
1
0
1
1
0
1
4
1
1
4
4


In [187]:
import pandas as pd 
test_img_data = os.listdir('./data/image_data/test')
test_id = []
i=0
for test_img_dir in test_img_data:
    test_id.append([test_img_dir.split(".")[0],res[i]])
    i = i +1

np_ids= np.array(test_id)
test_ids = pd.DataFrame(np_ids)
test_ids.columns = ['id', 'flower_class']
test_ids

,id,flower_class
0,0028624c49b3e0610ff9f1d111f5d532,4
1,002c30700185b7971369258b438070d5,1
2,00852f4f666acecd0c0d140365b42efd,4
3,00c08828fce04e360c732cac01edad9e,4
4,00d366e7877b6a78b104b57d67b60e6b,4
5,00e803f7bc6d21b6d6d3a98136ea4635,2
6,00e9cb1dca407810856e77b31309d5ab,1
7,014d33090eb706769ff782d8c500dc2a,4
8,015c8f0e6b95baf9dcbb34647624c5b8,4
9,0194948a29f0e891c54f88004fb4c51c,1


In [188]:
dir_data = './'
test_ids.to_csv(dir_data+'final_ans.csv', encoding='utf-8', index=False)